In [8]:
import sys
sys.path.append('../codes')

# 7. 함수 데커레이터와 클로저

__함수 데커레이터__ 는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해줍니다. 

데커레이터를 정확히 알기 위해 다음과 같은 내용을 먼저 알아야 합니다.

1. 파이썬이 데커레이터 구문을 평가하는 방식
2. 변수가 지역 변수인지 파이썬이 판단하는 방식
3. 클로저의 존재 이유와 작동 방식
4. nonlocal로 해결할 수 있는 문제

이 후에는 다음 주제를 심도 있게 다룰 수 있습니다.

1. 잘 작동하는 데커레이터 구현하기
2. 표준 라이브러리에서 제공하는 데커레이터
3. 매개변수화된 데커레이터 구현하기

## 7.1 데커레이터 기본 지식

데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)입니다.

데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블의 객체로 대체합니다. 

```
#1
@decorate
def target():
    print('running target()')
```

```
#2
def target():
    print('running target()')  
target = decorate(target)
```

`target()`은 `decorate(target)`이 반환된 함수를 가리키게 됩니다.

In [9]:
## ex 7.1
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
    
target()

running inner()


In [10]:
target

<function __main__.deco.<locals>.inner()>

데커레이터는 __편리 구문(syntatic sugar)__ 일 뿐입니다. 

> __syntatic sugar__
-  프로그래밍 언어차원에서 제공되는 논리적으로 간결하게 표현하는 것
- 중복되는 로직을 간결하게 표현하기 위함

런타임에 프로그램 행위를 변경하는 __메타프로그래밍__ 을 할 때 데커레이터가 상당히 편리합니다.

>__런타임__
    - 프로그램이 실행되고 있는 동안의 동작
__메타프로그래밍__
    - 자기 자신 혹은 다른 컴퓨터 프로그램을 데이터로 취급하여 프로그램을 작성-수정

1. __데커레이터는 데커레이트된 함수를 다른 함수로 대체__


2. __데커레이터는 모듈이 로딩될 때 바로 실행__

## 7.2 파이썬이 데커레이터를 실행하는 시점

데커레이터는 데커레이트된 함수가 정의된 직후, 즉 모듈을 로딩하는 시점(__임포트 타임__)에 실행됩니다.

In [11]:
## ex 7.2
registry = []
def register(func):
    print(f'running register{func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('register -> ', registry)
    f1()
    f2()
    f3()

running register<function f1 at 0x10de22680>
running register<function f2 at 0x10de224d0>


`register()`가 호출될 때 데커레이트 된 함수를 인수로 받으며 먼저 실행됩니다.

In [12]:
main()

running main()
register ->  [<function f1 at 0x10de22680>, <function f2 at 0x10de224d0>]
running f1()
running f2()
running f3()


모듈이 로딩된 후 `registery`는 데커레이트된 두 개의 함수(`f1`, `f2`)에 대한 참조를 가집니다.

In [13]:
import registration

running register<function f1 at 0x10decb200>
running register<function f2 at 0x10decb440>


In [14]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

데커레이터는 모듈이 임포트되자마자 실행되지만, 데커레이트된 함수는 명시적으로 호출될 때만 실행됨을 알 수 있습니다. 

이는 __임포트 타임__ 과 __런타임__ 의 차이를 명확히 보여줍니다.

1. 위 코드는 데커레이터 함수가 데커레이트되는 함수와 같은 모듈에 정의되어 있지만 일반적으로는 __데커레이터를 정의하는 모듈과 데커레이터를 적용하는 모듈을 분리해서 구현합니다.__


2. `register()` 데커레이터가 인수로 전달된 함수와 동일한 함수를 반환하지만, 일반적으로는 __데커레이터 내부 함수를 정의해서 반환합니다.__

## 7.3 데커레이터로 개선한 전략 패턴

In [15]:
## ex 7.3

promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    '''충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용'''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    '''20개 이상의 동일 상품을 구입하면 10% 할인 적용'''
    discount = 0
    for item in order.cart:
        if item.qunatity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    '''10종류 이상의 상품을 구입하면 전체 7% 할인 적용'''
    discount_items = {item.product for item in order.cart}
    if len(discount_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    '''최대로 할인받을 금액을 반환한다.'''
    return max(promo(order) for promo in promos)

1. 프로모션 전략 함수명이 특별한 형태로 되어 있을 필요 없습니다. 
2. `@promotion` 데커레이터는 데커레이트된 함수의 목적을 명확히 알려주며, 임시로 어떤 프로모션을 배제할 수 있습니다. 단지 데커레이터만 주석처리하면 됩니다.
3. 프로모션 할인 전략을 구현한 함수는 `@promotion` 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있습니다. 

데코레이트된 함수가 아닌 내부 함수를 사용하는 코드는 제대로 작동하기 위해 거의 항상 __클로저__에 의존합니다. 

## 7.4 변수 범위 규칙

In [16]:
## ex 7.4
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [17]:
b = 6
f1(3)

3
6


In [18]:
## ex 7.5
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

파이썬이 함수 본체를 컴파일할 때 `b`가 함수 안에서 할당되므로 `b`를 지역 변수로 판단합니다. 

함수 안에 할당하는 문장이 있지만 인터프리터가 `b`를 전역 변수로 다루기 원한다면, 다음과 같이 `global` 키워드를 이용해서 선언해야 합니다.

In [19]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [20]:
b

9

In [21]:
f3(3)

3
9


In [22]:
b = 30
b, f3(3)

3
30


(30, None)

## 7.5 클로저

클로저는 내포된 함수 안에서만 의미가 있지만 익명함수와는 다른 개념입니다.

클로저는 __함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수__입니다. 함수가 익명 함수인지 여부는 중요하지 않습니다. __함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요합니다.__

In [23]:
## ex 7.8
class Average():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [24]:
avg = Average()
avg(10)

10.0

In [25]:
avg(11)

10.5

In [26]:
avg(12)

11.0

함수는 호출되면 `average()` 함수 객체를 반환합니다. `average()` 함수는 호출될 때마다 받은 인수를 `series` 리스트에 추가하고 현재까지의 평균을 계산해서 출력합니다.

`ex 7.8`을 보면 클래스로 생성된 객체가 데이터를 보관하는 방법은 명확히 알 수 있습니다. `self.series` 객체 속성에 저장되기 때문입니다. 

In [27]:
## ex 7.9
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [28]:
avg = make_averager()
avg(10)

10.0

In [29]:
avg(11)

10.5

In [30]:
avg(12)

11.0

하지만 `ex 7.9`을 보면 `series`를 어디서 찾을까요 ?

`make_average()` 함수 본체 안에서 `series = []`로 초기화하고 있으므로 `series`는 이 함수의 지역 변수입니다. `avg(10)`을 호출할 때, `make_averager()`함수는 이미 반환했으므로 지역 범위도 이미 사라진 후입니다. 

그 후 `average` 안에 있는 `series`는 __자유 변수(free variable)__입니다. 자유 변수란 __지역 범위에 바인딩되어 있지 않은 변수를 의미합니다.__ 

![ex7.1](../images/7_1.png)

> __바인딩__
    - 변수에 변수와 관련된 속성을 연관시키는 것

반환된 `averager()`객체를 조사해보면 파이썬이 컴파일된 함수 본체를 나타내는 `__code__`속성 안에 어떻게 지역 변수와 자유 변수의 '이름'을 저장하는지 알 수 있습니다.

In [31]:
avg.__code__.co_varnames

('new_value', 'total')

In [32]:
avg.__code__.co_freevars

('series',)

`series`에 대한 바인딩은 반환된 `avg()`함수의 `__closure__` 속성에 저장됩니다.

In [33]:
avg.__closure__

(<cell at 0x10ddfebd0: list object at 0x10dd4ed70>,)

이는 `avg.__code__.co_freevars`의 이름에 대응됩니다.

이 항목은 cell 객체이며 이 객체의 항목은 `cell_contents`속성에서 찾을 수 있습니다.

In [34]:
avg.__closure__[0].cell_contents

[10, 11, 12]

> __cell 객체__
    - 《셀》 객체는 여러 스코프에서 참조하는 변수를 구현하는 데 사용됩니다. 이러한 변수마다, 값을 저장하기 위해 셀 객체가 만들어집니다; 값을 참조하는 각 스택 프레임의 지역 변수에는 해당 변수를 사용하는 외부 스코프의 셀에 대한 참조가 포함됩니다. 값에 액세스하면, 셀 객체 자체 대신 셀에 포함된 값이 사용됩니다. 이러한 셀 객체의 역참조(de-referencing)는 생성된 바이트 코드로부터의 지원이 필요합니다; 액세스 시 자동으로 역참조되지 않습니다. 셀 객체는 다른 곳에 유용하지는 않습니다.

1. __클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수__


2. __함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근 가능__


3. __함수가 '비전역' 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에서 정의된 경우 뿐이라는 점에 주의__

## 7.6 nonlocal 선언

In [35]:
## ex 7.13
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [36]:
avg = make_averager()

In [37]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

`count += 1` 은 `count = count + 1`을 의미하는데 `averager` 본체 안에서 `count` 변수에 할당하고 있기 때문에 `count`를 지역 변수로 만들어서 에러가 발생합니다.

앞선 예제에서는 `series`를 변수에 할당하지 않기 때문에 아무런 문제도 발생하지 않았습니다.

하지만 숫자, 문자열, 튜플 등 불변형은 읽을 수만 있고 값을 갱신할 수 없으므로 재할당을 하면서 `count`라는 지역 변수를 생성하게 되고 자유 변수가 아니게 되므로 클로저에 저장되지 않습니다.

In [38]:
## ex 7.14
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager

In [39]:
avg = make_averager()

In [40]:
avg(10)

10.0

In [41]:
avg(11)

10.5

In [42]:
avg(12)

11.0

이를 해결하기 위해선 `nonlocal` 선언을 해주면 됩니다. 변수를 `nonlocal` 선언하면, 함수 안에서 재할당을 하더라도 그 변수는 자유변수로 남아있을 수 있습니다.

새로운 값을 nonlocal 변수에 할당하면 클로저에 저장된 바인딩이 변경됩니다.

In [43]:
avg.__code__.co_freevars

('count', 'total')

## 7.7 간단한 데커레이터 구현하기

In [44]:
! cat ../codes/clockdeco.py

import time

def clock(func):
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed,
                                         name,
                                         arg_str,
                                         result))
        return result
    return clocked


In [45]:
! cat ../codes/clockdeco_demo.py

import time
from clockdeco import clock

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print(' 6! = ', factorial(6))


In [46]:
! python3 ../codes/clockdeco_demo.py

**************************************** Calling snooze(.123)
[0.12340668s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000180s] factorial(1) -> 1
[0.00017545s] factorial(2) -> 2
[0.00019098s] factorial(3) -> 6
[0.00020355s] factorial(4) -> 24
[0.00021607s] factorial(5) -> 120
[0.00023125s] factorial(6) -> 720
 6! =  720


파이썬 인터프리터가 내부적으로 `clocked()`를 `factorial`에 할당했습니다. 

In [47]:
import clockdeco_demo
clockdeco_demo.factorial.__name__

'clocked'

하지만 위 예제는 몇 가지 단점이 존재합니다. 키워드 인수를 지원하지 않고, 데커레이트된 함수의 `__name__`, `__doc__`를 씹습니다. 

`functools.wraps()`데커레이트드를 이용해서 `func`에서 `clocked`로 관련된 속성을 복사합니다. 

In [48]:
! cat ../codes/clockdeco2.py

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed,
                                         name,
                                         arg_str,
                                         result))
        return result
    return clocked


In [49]:
from clockdeco2 import clock

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial.__name__

'factorial'